In [200]:
import pandas as pd
import plotly.express as px
from data import var_f4
from datetime import datetime
from general import ord_num, unif_colors
pd.set_option('display.max_columns', 50)

In [201]:
f4 = pd.read_csv(var_f4['path_df'] + "220616_f4" +".csv", sep=';', dtype=str)
f4.linea = f4.linea.str.capitalize()
f4.descripcion_linea = f4.descripcion_linea.str.capitalize()
f4[var_f4["costo"]] = pd.to_numeric(f4[var_f4["costo"]])
f4["linea_graf"] = f4["linea"] + " - " + f4["descripcion_linea"]

In [202]:
corte = datetime.strptime("2021-06-16", "%Y-%m-%d")
f4.loc[:, var_f4["fechas"]] = f4[var_f4["fechas"]].apply(lambda x: x.replace(["ene", "abr", "ago", "dic"], ["jan", "apr", "aug", "dec"], regex=True))
for i in var_f4["fechas"]: f4[i] = pd.to_datetime(f4[i])
f4_db = f4.loc[f4.tipo_redinv == "dado de baja"]
f4_2021 = f4_db.loc[f4_db.fecha_reserva <= corte]
f4_2022 = f4_db.loc[f4_db.fecha_reserva >= "2022-01-01"]


In [203]:
f4_2021["Año"] = "2021" + "-" + corte.strftime(format = "%b-%d")
f4_2022["Año"] = "2022" + "-" + corte.strftime(format = "%b-%d")


C:\Users\EXT_MA~1\AppData\Local\Temp/ipykernel_23832/1746822602.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\EXT_MA~1\AppData\Local\Temp/ipykernel_23832/1746822602.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [204]:
df_f4 = pd.concat([f4_2021,f4_2022])

In [215]:
group_f4_linea = df_f4.groupby(["linea_graf","Año"])["total_precio_costo"].sum().reset_index()
# group_f4_linea.to_excel("220616_linea_x_años.xlsx", index=False)

In [206]:
ord_x = ord_num(group_f4_linea,'linea_graf',var_f4['costo'])
top_5 = ord_num(group_f4_linea,'linea_graf',var_f4['costo'])[0:10]
colors = unif_colors(group_f4_linea, "Año")

In [207]:
f4_linea_top_10 = group_f4_linea.loc[group_f4_linea.linea_graf.isin(top_5)]


In [220]:
graph_linea_x_años = px.bar(f4_linea_top_10, x="linea_graf", y = "total_precio_costo", color = "Año", barmode= "group", category_orders ={"linea_graf":ord_x},text_auto = ".2s"  , text = "total_precio_costo", title = "F4 dado de baja para top 10 de linea 2021 vs 2022", color_discrete_map = colors, labels = {"total_precio_costo":"Costo","linea_graf":"Línea"}  ) 
graph_linea_x_años.update_layout(legend = dict(yanchor = 'top', y = 0.95, xanchor = 'left', x = 0.7),font =dict(size = 15), margin=dict(l=0,r=10,b=0))
graph_linea_x_años.write_image("F4_linea_años.png",height = 500,  width = 1000)